Программа повышения квалификации (научно-педагогических) работников НИУ ВШЭ

# Python для исследователей

*Автор: Алла Тамбовцева, НИУ ВШЭ*  

## Web-scraping: скрэйпинг новостного сайта

Мы уже немного познакомились со структурой html-файлов, теперь попробуем выгрузить информацию из реальной страницы, а точнее, с реального сайта [nplus1.ru](https://nplus1.ru/).

**Наша задача:** выгрузить недавние новости в датафрейм `pandas`, чтобы потом сохранить все в csv-файл.

Сначала сгрузим весь html-код страницы и сохраним его в отдельную переменную. Для этого нам понадобится библиотека `requests`. Импортируем её:

In [1]:
import requests

Сохраним ссылку на главную страницу сайта в переменную `url` для удобства и выгрузим страницу. (Разумеется, это будет работать при подключении к интернету. Если соединение будет отключено, Python выдаст `NewConnectionError`).

In [2]:
url = 'https://nplus1.ru/' # сохраняем
page = requests.get(url) # загружаем страницу по ссылке

Если мы просто посмотрим на объект, мы ничего особенного не увидим:

In [3]:
page  # response 200 - страница загружена

<Response [200]>

Импортируем функцию `BeautifulSoup` из библиотеки `bs4` (от *beautifulsoup4*) и заберём со страницы `page` код html в виде текста. 

In [3]:
from bs4 import BeautifulSoup  # не спрашивайте, почему BeautifulSoup

In [4]:
soup = BeautifulSoup(page.text, 'lxml')

Если выведем `soup` на экран, мы увидим то же самое, что в режиме разработчика или в режиме происмотра исходного кода (`view-source` через *Ctrl+U* в Google Chrome).

In [ ]:
# soup

Для просмотра выглядит не очень удобно.  «Причешем» наш `soup` – воспользуемся методом `.prettify()` в сочетании с функцией `print()`.

In [ ]:
# print(soup.prettify())

В такой выдаче ориентироваться гораздо удобнее (но при желании, то же можно увидеть в браузере, на большом экране).

Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`. Посмотрим на кусочки кода, соответствующие всем ссылкам на главной странице сайта:

In [6]:
for link in soup.find_all('a'):
    print(link.get('href'))

#
/
#
#
/rubric/astronomy
/rubric/physics
/rubric/biology
/rubric/robots-drones
/theme/explainatorium
/theme/bookshelf
/theme/Courses
/theme/coronavirus-history
/
#
/rubric/astronomy
/rubric/physics
/rubric/biology
/rubric/robots-drones
#
/theme/explainatorium
/theme/bookshelf
/theme/Courses
/theme/coronavirus-history
https://nplus1.ru/blog/2020/04/02/marsphone
https://nplus1.ru/blog/2020/04/02/marsphone
https://nplus1.ru/blog/2020/04/02/low
https://nplus1.ru/blog/2020/03/23/pompeiis-best-houses
https://nplus1.ru/blog/2020/03/22/infodemia
https://nplus1.ru/blog/2020/03/17/isolation-and-iss
https://nplus1.ru/blog/2020/03/16/ludere-ridere-occest-vivere
https://nplus1.ru/blog/2020/03/12/onemoredelay
https://nplus1.ru/blog/2020/03/12/public-opinion-yudin
https://nplus1.ru/blog/2020/03/06/death-from-the-skies
https://nplus1.ru/blog/2020/04/06/maps-of-meaning
/news/2020/04/08/three-fireballs-europe
/news/2020/04/08/apple-patent
/news/2020/04/08/modernize
/news/2020/04/08/no-chronic-diseases


Ссылок много. Но нам нужны только новости – ссылки, которые начинаются со слова `/news`. Добавим условие: будем выбирать только те ссылки, в которых есть `/news`. Создадим пустой список `urls` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [5]:
urls = []

for link in soup.find_all('a'):
    if '/news' in link.get('href'):
        urls.append(link.get('href'))
urls

['/news/2020/06/01/parenting-cognition',
 '/news/2020/06/01/vtol',
 '/news/2020/06/01/early-postmortem-interval',
 '/news/2020/06/01/prospective-memory-and-procrastination',
 '/news/2020/06/01/gray-hair',
 '/news/2020/06/01/jet-fuel',
 '/news/2020/06/01/robusta-coffee-warming-stress',
 '/news/2020/06/01/transport',
 '/news/2020/05/31/bh-outburst',
 '/news/2020/05/31/crew-dragon-docking',
 '/news/2020/05/30/crew-dragon-first-flight',
 '/news/2020/05/30/sans-forgetica',
 '/news/2020/05/30/sex-on-hippocampus',
 '/news/2020/05/30/ceramic-ris-effect',
 '/news/2020/05/30/starship-fail-again',
 '/news/2020/05/30/westerlund-2-hubble',
 '/news/2020/05/29/exoskeleton-gloves',
 '/news/2020/05/29/ancient-millipede',
 '/news/2020/05/29/hahaha',
 '/news/2020/05/25/QND-metamaterial',
 '/news/2020/05/28/isolde-radioactive-molecule',
 '/news/2020/05/26/bn-graphene-valleytronics',
 '/news/2020/05/29/meta-helmeta',
 '/news/2020/05/26/moral-choice-machine',
 '/news/2020/05/26/black-hole-flickering']

Ссылки, которые у нас есть в списке `urls`, относительные: они неполные, начало ссылки (название сайта) отсутствует. Давайте превратим их в абсолютные ‒ склеим с ссылкой https://nplus1.ru.

In [6]:
full_urls = []

for u in urls:
    res = 'https://nplus1.ru' + u
    full_urls.append(res) 

full_urls

['https://nplus1.ru/news/2020/06/01/parenting-cognition',
 'https://nplus1.ru/news/2020/06/01/vtol',
 'https://nplus1.ru/news/2020/06/01/early-postmortem-interval',
 'https://nplus1.ru/news/2020/06/01/prospective-memory-and-procrastination',
 'https://nplus1.ru/news/2020/06/01/gray-hair',
 'https://nplus1.ru/news/2020/06/01/jet-fuel',
 'https://nplus1.ru/news/2020/06/01/robusta-coffee-warming-stress',
 'https://nplus1.ru/news/2020/06/01/transport',
 'https://nplus1.ru/news/2020/05/31/bh-outburst',
 'https://nplus1.ru/news/2020/05/31/crew-dragon-docking',
 'https://nplus1.ru/news/2020/05/30/crew-dragon-first-flight',
 'https://nplus1.ru/news/2020/05/30/sans-forgetica',
 'https://nplus1.ru/news/2020/05/30/sex-on-hippocampus',
 'https://nplus1.ru/news/2020/05/30/ceramic-ris-effect',
 'https://nplus1.ru/news/2020/05/30/starship-fail-again',
 'https://nplus1.ru/news/2020/05/30/westerlund-2-hubble',
 'https://nplus1.ru/news/2020/05/29/exoskeleton-gloves',
 'https://nplus1.ru/news/2020/05/29/

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `full_urls` в цикле. Посмотрим на новость с индексом 1, у вас может быть другая, новости обновляются.

In [7]:
url0 = full_urls[1]

page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text, 'lxml')

В коде каждой страницы с новостью есть часть с мета-информацией: датой, именем автора и проч. Такая информация окружена тэгом `<meta></meta>`. Посмотрим:

In [8]:
soup0.find_all('meta')

[<meta charset="utf-8"/>,
 <meta content="ie=edge" http-equiv="x-ua-compatible"/>,
 <meta content="width=device-width, initial-scale=1" name="viewport"/>,
 <meta content="yes" name="apple-mobile-web-app-capable"/>,
 <meta content="black" name="apple-mobile-web-app-status-bar-style"/>,
 <meta content="2020-06-01" itemprop="datePublished"/>,
 <meta content="Василий Сычев" name="mediator_author"/>,
 <meta content="Новые аппараты должны будут заменить ScanEagle самолетного типа" name="description"/>,
 <meta content="Василий Сычев" name="author"/>,
 <meta content="" name="copyright"/>,
 <meta content="Для ВМС США разработают тактические палубные беспилотники" property="og:title"/>,
 <meta content="https://nplus1.ru/images/2020/06/01/be90f15b4481152166460e677a499d32.jpg" property="og:image"/>,
 <meta content="https://nplus1.ru/news/2020/06/01/vtol" property="og:url"/>,
 <meta content="Новые аппараты должны будут заменить ScanEagle самолетного типа" property="og:description"/>,
 <meta content

Из этого списка нам нужны части с именем автора, датой, заголовком и кратким описанием. Воспользуемся поиском по атрибуту `name`. Передадим функции `find_all()` в качестве аргумента словарь с названием и значением атрибута: 

In [9]:
soup0.find_all('meta', {'name' : 'author'}) # например, автор

[<meta content="Василий Сычев" name="author"/>]

Теперь выберем единственный элемент полученного списка (с индексом 0):

In [10]:
soup0.find_all('meta', {'name' : 'author'})[0]

<meta content="Василий Сычев" name="author"/>

Нам нужно вытащить из этого объекта `content` – имя автора. Посмотрим на атрибуты:

In [11]:
soup0.find_all('meta', {'name' : 'author'})[0].attrs

{'name': 'author', 'content': 'Василий Сычев'}

Как получить отсюда `content`? Очень просто, ведь это словарь! А доставать из словаря значение по ключу мы умеем.

In [12]:
author = soup0.find_all('meta', {'name' : 'author'})[0].attrs['content']
author

'Василий Сычев'

Аналогичным образом извлечем дату, заголовок и описание.

In [13]:
date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].attrs['content']
title = soup0.find_all('meta', {'property' : 'og:title'})[0].attrs['content']
description = soup0.find_all('meta', {'name' : 'description'})[0].attrs['content']

Осталось вытащить рубрики и сложность текста. Если мы посмотрим на исходный код страницы, мы увидим, что нужная нам информация находится в тэгах `<p></p>`:

In [14]:
soup0.find_all('p')

[<p class="table">
 <a data-rubric="weapon" href="/rubric/weapon">Оружие</a>
 </p>, <p class="table">
 <a href="/news/2020/06/01">
 <time content="2020-06-01" data-unix="1591017919" itemprop="datePublished">
 <span>16:25</span>
 <span>01 Июнь 2020</span>
 </time>
 </a>
 </p>, <p class="table">
 <a href="/difficult/3.1">
 <span>Сложность</span>
 <span class="difficult-value">3.1</span>
 </a>
 </p>, <p class="title">V-Bat</p>, <p class="credits">MartinUAV</p>, <p>Агентство перспективных оборонных разработок Пентагона (DARPA) и специалисты Исследовательского управления ВМС США в ближайшее время дадут старт программе разработки нового тактического палубного беспилотного летательного аппарата, пригодного к взлету и посадки на палубу практического любого корабля в составе флота. Как пишет <a href="http://aviationweek.com" rel="nofollow" target="_blank">Aviation Week</a>, новые аппараты должны будут заменить разведывательные беспилотники ScanEagle, запускаемые с помощью пневматических катапул

Выберем из полученного списка первый элемент и найдем в нем все тэги `<a>`:

In [15]:
soup0.find_all('p')[0].find_all('a')

[<a data-rubric="weapon" href="/rubric/weapon">Оружие</a>]

Получился список из одного элемента. Применим списковые включения – вытащим из каждого элемента (их могло бы быть больше) текст и поместим его в новый список `rubrics`.

In [16]:
rubrics = [r.text for r in soup0.find_all('p')[0].find_all('a')]
rubrics

['Оружие']

Осталась только сложность. Возьмем соответствующий кусок кода:

In [17]:
soup0.find_all('span', {'class' : 'difficult-value'})

[<span class="difficult-value">3.1</span>]

И выберем оттуда текст.

In [18]:
diff = soup0.find_all('span', {'class' : 'difficult-value'})[0].text
diff

'3.1'

Теперь перейдем к тексту самой новости. Как можно заметить, текст сохранен в абзацах `<p></p>`, причем безо всяких атрибутов. Сообщим Python, что нас интересуют куски с пустым атрибутом `class`:

In [19]:
text_list = soup0.find_all('p', {'class' : None})

«Выцепим» все тексты (без тэгов) из полученного списка:

In [20]:
text = [t.text for t in text_list]

Склеим все элементы списка `text` через пробел:

In [21]:
final_text = ' '.join(text)
final_text

'Агентство перспективных оборонных разработок Пентагона (DARPA) и\xa0специалисты Исследовательского управления ВМС США в\xa0ближайшее время дадут старт программе разработки нового тактического палубного беспилотного летательного аппарата, пригодного к\xa0взлету и\xa0посадки на\xa0палубу практического любого корабля в\xa0составе флота. Как пишет Aviation Week, новые аппараты должны будут заменить разведывательные беспилотники ScanEagle, запускаемые с\xa0помощью пневматических катапульт и\xa0требующие специального аэрофинишера для завершения полета. По\xa0итогам разработки нового беспилотника, военные рассчитывают получить аппарат массой до\xa090,7\xa0— 136\xa0килограммов, способный нести полезную нагрузку массой до\xa027\xa0килограммов. Беспилотник должен быть способен на\xa0вертикальные взлет и\xa0посадку, но\xa0при этом его крейсерский полет должен проходить в\xa0самолетном режиме. Благодаря этому военные и\xa0рассчитывают избавиться от\xa0катапульты и\xa0аэрофинишера. Новые аппараты 

Все здорово, только мешают отступы-переходы на новую строку `\n`. Заменим их на пробелы с помощью метода `.replace`:

In [22]:
final_text = final_text.replace('\n', ' ')

Теперь осталось совсем чуть-чуть. Написать готовую функцию для всех проделанных нами действий и применить ее в цикле для всех ссылок в списке `full_urls`. Напишем! Аргументом функции будет ссылка на новость, а возвращать она будет текст новости и всю необходимую информацию (дата, автор, сложность и проч.). Скопируем все строки кода выше.

In [23]:
def GetNews(url0):
    """
    Returns a tuple with url0, date, author, description, title, final_text, rubrics, diff.
    Parameters:
    
    url0 is a link to the news (string)
    """
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text, 'lxml')
    author = soup0.find_all('meta', {'name' : 'author'})[0].attrs['content']
    date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].attrs['content']
    title = soup0.find_all('meta', {'property' : 'og:title'})[0].attrs['content']
    description = soup0.find_all('meta', {'name' : 'description'})[0].attrs['content']
    rubrics = [r.text for r in soup0.find_all('p')[0].find_all('a')]
    diff = soup0.find_all('span', {'class' : 'difficult-value'})[0].text
    text_list = soup0.find_all('p', {'class' : None})
    text = [t.text for t in text_list]
    final_text = ' '.join(text)
    final_text = final_text.replace('\n', ' ')
    
    return url0, date, author, description, title, final_text, rubrics, diff

Уфф. Осталось применить ее в цикле. Но давайте не будем спешить: импортируем функцию `sleep` для задержки, чтобы на каждой итерации цикла, прежде чем перейти к следующей новости, Python ждал несколько секунд. Во-первых, это нужно, чтобы сайт «не понял», чтобы мы его грабим, да еще автоматически. Во-вторых, с небольшой задержкой всегда есть гарантия, что страница прогрузится (сейчас это пока не очень важно, но особенно актуально будет, когда будем обсуждать встраивание в браузер с Selenium). Приступим.

In [24]:
from time import sleep

In [25]:
news = [] # это будет список из кортежей, в которых будут храниться данные по каждой новости

for link in full_urls:
    res = GetNews(link)
    news.append(res)
    sleep(3) # задержка в 3 секунды

Так теперь выглядит первый элемент списка:

In [26]:
news[0]

('https://nplus1.ru/news/2020/06/01/parenting-cognition',
 '2020-06-01',
 'Алиса Бахарева',
 'Такая зависимость может быть универсальной для всех животных',
 'Долгое детство и забота родителей помогли врановым стать умнее',
 'Ученые предложили модель влияния внешних факторов на развитие когнитивных способностей животных, в которой центральную роль играет детство и забота родителей. Свою гипотезу авторы работы, опубликованной в журнале Philosophical Transactions of the Royal Society B: Biological Sciences, иллюстрируют двумя примерами: новокаледонские вороны в течение года остаются под опекой родителей, чтобы научаться изготавливать инструменты; кукши, которым приходится расти в чужой семье, теряют возможность обучаться поиску добычи и защите от хищников. Когнитивные способности человека уникальны, и\xa0основой их\xa0развития является долгое детство\xa0— период пластичности, в\xa0который люди осваивают навыки и\xa0развивают познавательный потенциал. Однако долгое взросление свойственно 

Импортируем `pandas` и создадим датафрейм из списка кортежей: 

In [30]:
import pandas as pd

In [31]:
df = pd.DataFrame(news)

In [32]:
df.head(2)

,0,1,2,3,4,5,6,7
0,https://nplus1.ru/news/2020/06/01/parenting-co...,2020-06-01,Алиса Бахарева,Такая зависимость может быть универсальной для...,Долгое детство и забота родителей помогли вран...,Ученые предложили модель влияния внешних факто...,"[Зоология, Нейронауки]",2.6
1,https://nplus1.ru/news/2020/06/01/vtol,2020-06-01,Василий Сычев,Новые аппараты должны будут заменить ScanEagle...,Для ВМС США разработают тактические палубные б...,Агентство перспективных оборонных разработок П...,[Оружие],3.1


Переименуем столбцы в базе.

In [33]:
df.columns = ['link', 'date', 'author', 'desc', 'title', 'text', 'rubric', 'diffc']

In [34]:
df.head(2)

,link,date,author,desc,title,text,rubric,diffc
0,https://nplus1.ru/news/2020/06/01/parenting-co...,2020-06-01,Алиса Бахарева,Такая зависимость может быть универсальной для...,Долгое детство и забота родителей помогли вран...,Ученые предложили модель влияния внешних факто...,"[Зоология, Нейронауки]",2.6
1,https://nplus1.ru/news/2020/06/01/vtol,2020-06-01,Василий Сычев,Новые аппараты должны будут заменить ScanEagle...,Для ВМС США разработают тактические палубные б...,Агентство перспективных оборонных разработок П...,[Оружие],3.1


Теперь внесем изменения: сделаем столбец `diffc` числовым – типа *float*.

In [35]:
df['diffc'] = [float(i) for i in df.diffc]

Теперь сложность представлена в базе как количественный показатель, и описывать ее можно соответствующим образом:

In [36]:
df.diffc.describe()

count    25.000000
mean      3.724000
std       1.731637
min       1.300000
25%       2.500000
50%       3.100000
75%       5.300000
max       8.100000
Name: diffc, dtype: float64

Объединим рубрики в *text* в одну строку через запятую:

In [38]:
df['rubric'] = [','.join(r) for r in df.rubric]

Давайте почистим текст новостей – уберем оттуда текст, не относящийся к новостям. Найдем лишнее:

In [39]:
df.text[0]

'Ученые предложили модель влияния внешних факторов на развитие когнитивных способностей животных, в которой центральную роль играет детство и забота родителей. Свою гипотезу авторы работы, опубликованной в журнале Philosophical Transactions of the Royal Society B: Biological Sciences, иллюстрируют двумя примерами: новокаледонские вороны в течение года остаются под опекой родителей, чтобы научаться изготавливать инструменты; кукши, которым приходится расти в чужой семье, теряют возможность обучаться поиску добычи и защите от хищников. Когнитивные способности человека уникальны, и\xa0основой их\xa0развития является долгое детство\xa0— период пластичности, в\xa0который люди осваивают навыки и\xa0развивают познавательный потенциал. Однако долгое взросление свойственно и другим животным: летучим мышам, китам, слонам и\xa0некоторым птицам. Дает\xa0ли долгое детство преимущество в\xa0когнитивном развитии для этих видов так\xa0же, как и\xa0для человека, остается неясным.  Ученые из\xa0Великобр

Лишний текст находится после фразы 'Нашли опечатку?'. Так давайте будем разбивать строки по этой фразе с помощью метода `.split()` и брать все, что до нее (элемент с индексом 0).

In [41]:
df['clean_text'] = [t.split('Нашли опечатку?')[0] for t in df.text]

Осталось только заменить непонятные символы `\xa0` на пробелы:

In [41]:
df['clean_text'] = [t.replace("\xa0", " ") for t in df.clean_text]

In [42]:
df.clean_text[0]

'В ночь с 1 на 2 апреля над Центральной Европой в течение трех часов пролетели три болида, причем два из них — за один час. Об этом сообщает Международная метеорная организация (International Meteor Organization, IMO). По разным оценкам в атмосферу Земли за день попадает от 5 до 300 тонн космической пыли. Сгорающие в верхних слоях частицы, оставляющие светящийся след, называют метеорами. Очень яркие метеоры, или болиды, возникают, когда в атмосферу попадает тело метровых размеров и больше — яркость вспышки в таком случае превышает яркость Венеры. Как правило, такие метеоры — одиночное явление.  Теперь же IMO сообщила сразу о трех таких событиях, произошедших за короткий промежуток времени. Два болида — метеора, более ярких, чем Венера, — попали в объектив камеры проекта CAMS NASA, которая находится в бельгийском городе Графонтен. Они также были зафиксированы камерами сети FRIPON, которая объединяет более 100 камер полного неба и приемников радиосигнала на территории Франции, Бельгии и 

Всё! Сохраняем датафрейм в файл. Для разнообразия сохраним в Excel:

In [43]:
df.to_excel('nplus-news.xlsx')